In [6]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

## 데이터 컬럼 설명
'''
no : 차량고유번호
cartype : '차량타입'
receipttime : '예정일시'
settime : '배차일시'
ridetime : '승차일시'
startpos1 : '출발지구군'
startpos2 : '출발지상세'
endpos1 : '목적지구군'
endpos2 : '목적지상세'
'''

## 요청 인자
key = '474d574475686e753130366a52454b54'
file_type = 'xml' # xml - xml, xls - 엑셀, json - json
service = 'disabledCalltaxi' # 장애인콜시스템
start_index = 1 # 데이터 행 시작번호
end_index = 100 # 데이터 행 끝번호
reg_date = '20210901' # 요청일

## 요청 url
url = f'http://openapi.seoul.go.kr:8088/{key}/{file_type}/{service}/{start_index}/{end_index}/{reg_date}'

## API 요청
html = requests.get(url)
soup = bs(html.text)

## parsing
data = []
items = soup.findAll('item')
for item in items:
    no = item.find('no').text
    cartype = item.find('cartype').text
    receipttime = item.find('receipttime').text
    settime = item.find('settime').text
    ridetime = item.find('ridetime').text
    startpos1 = item.find('startpos1').text
    startpos2 = item.find('startpos2').text
    endpos1 = item.find('endpos1').text
    endpos2 = item.find('endpos2').text

    data.append([no, cartype, receipttime, settime, ridetime, startpos1, startpos2, endpos1, endpos2])

## list => dataframe
df = pd.DataFrame(data, columns=['no', 'cartype', 'receipttime', 'settime', 'ridetime', 'startpos1', 'startpos2', 'endpos1', 'endpos2'])

## string column to datetime column
df['receipttime'] = pd.to_datetime(df['receipttime'].map(lambda x: x.replace('오전', 'AM') if '오전' in x else x.replace('오후', 'PM')), format = '%Y-%m-%d %p %I:%M:%S')
df['settime'] = pd.to_datetime(df['settime'].map(lambda x: x.replace('오전', 'AM') if '오전' in x else x.replace('오후', 'PM')), format = '%Y-%m-%d %p %I:%M:%S')
df['ridetime'] = pd.to_datetime(df['ridetime'].map(lambda x: x.replace('오전', 'AM') if '오전' in x else x.replace('오후', 'PM')), format = '%Y-%m-%d %p %I:%M:%S')

## split date & time function
def split_column(df, col_name):
    df[f'{col_name}_date'] = df[col_name].dt.date
    df[f'{col_name}_time'] = df[col_name].dt.time
    df = df.drop([col_name], axis=1)
    return df

## split data
for col_name in ['receipttime', 'settime', 'ridetime']:
    df = split_column(df, col_name)

## save dataframe
df.to_csv(f'{start_index}_{end_index}_{reg_date}.csv', encoding='cp949', index=None)

C:\python\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
